In [23]:
import pandas as pd
import numpy as np

In [28]:
file_path = "nieruchomosci/geocoded/nieruchomosci_online_geocoded.csv"
df = pd.read_csv(file_path, sep="\t")
df.shape

(27825, 39)

In [25]:
df.drop_duplicates(inplace=True)

In [30]:
df[df.duplicated()].shape

(27, 39)

In [26]:
df.shape

(27798, 39)

In [4]:
df["Miasto"] = df["Miasto"].str.lower()

In [5]:
df["Cena"] = df["Cena"].str[:-3]
df["Cena"] = df["Cena"].str.replace(",", ".")
df["Cena"] = df["Cena"].str.replace(" ", "")
df["Cena"] = df["Cena"].astype(float)

In [6]:
df["Powierzchnia"] = df["Powierzchnia"].str[:-3]
df["Powierzchnia"] = df["Powierzchnia"].str.replace(",", ".")
df["Powierzchnia"] = df["Powierzchnia"].str.replace(" ", "")
df["Powierzchnia"] = df["Powierzchnia"].astype(float)

In [7]:
from typing import Optional

def convert_string_to_floor(floor: str, max_floor: Optional[str] = None):
    if floor == "parter":
        return 0
    elif floor == "suterena":
        return -1
    elif floor == "poddasze":
        return int(max_floor)
    else:
        return int(floor)

def prepare_floor(row: pd.Series):
    floor = row["Pietro"]
    floor_elements = floor.split("/")
    if len(floor_elements) == 2:
        your_floor = floor_elements[0].strip()
        max_floor = floor_elements[1].strip()
        your_floor = convert_string_to_floor(your_floor, max_floor)
        max_floor = int(max_floor)
           
    if len(floor_elements) == 1:
        your_floor = floor_elements[0].strip()
        if your_floor == "-":
            your_floor = None
            max_floor = None
        else:
            your_floor = convert_string_to_floor(your_floor)
        max_floor = None
    
    row["pietro mieszkania"] = your_floor
    row["maksymalne pietro"] = max_floor
    return row
    
df = df.apply(prepare_floor, axis=1)

In [8]:
df["polozenie mieszkania"] = df["pietro mieszkania"] / df["maksymalne pietro"]

In [9]:
df["Liczba pokoi"].replace({"-": np.nan}, inplace=True)
df["Liczba pokoi"] = df["Liczba pokoi"].astype(float)

In [10]:
df["Stan mieszkania"].replace({
    "bardzo dobry": "do zamieszkania", 
    "deweloperski": "do remontu", 
    "do odswiezenia": "do remontu",
    "dobry": "do zamieszkania", 
    "nowe wykonczone": "do zamieszkania",
    "wysoki standard": "do zamieszkania",
    "-": "Brak danych"
}, inplace=True)
df.rename(columns = {"Stan mieszkania": "Stan wykonczenia"}, inplace=True)

In [11]:
df.loc[~df["winda"].isna(), "winda"] = 1
df.loc[df["winda"].isna(), "winda"] = 0

df.rename(columns = {"winda": "Winda"}, inplace=True)
df["Winda"] = df["Winda"].astype(float)

In [12]:
df.loc[~df["Powierzchnia dodatkowa"].isna(), "Powierzchnia dodatkowa"] = 1
df.loc[df["Powierzchnia dodatkowa"].isna(), "Powierzchnia dodatkowa"] = 0

In [13]:
parking_slot_values = [
    "-",
    "przynalezne na ulicy (1 miejsce parkingowe)",
    "mozliwosc wykupienia, parking publiczny / na ulicy",
    "mozliwosc wykupienia, parking publiczny / na ulicy",
    "parking strzezony w poblizu",
    "parking strzezony w poblizu, parking publiczny / na ulicy",
]
df.loc[~df["Miejsce parkingowe"].isin(parking_slot_values), "Miejsce parkingowe"] = 1
df.loc[df["Miejsce parkingowe"].isin(parking_slot_values), "Miejsce parkingowe"] = 0

In [14]:
df.loc[df["Rynek"].isna(), "Rynek"] = None
df.loc[df["Rynek"].str.contains("pierwotny", na=False), "Rynek"] = "pierwotny"

In [15]:
df.loc[df["Rok budowy"].isna(), "Rok budowy"] = None
df.loc[df["Rok budowy"] == "-", "Rok budowy"] = None

In [16]:
df["Czynsz"] = df["Czynsz"].str.replace("zl", "")
df["Czynsz"] = df["Czynsz"].str.replace(",", ".")
df["Czynsz"] = df["Czynsz"].str.replace(" ", "")
df.loc[(df["Czynsz"].isna()), "Czynsz"] = np.nan
df["Czynsz"] = df["Czynsz"].astype(float)

In [17]:
df.drop(columns=[
    "Cena za metr", 
    "Tytul", 
    "Adres",
    "Typ oferty",
    "Dostepne",
    "Charakterystyka mieszkania",
    "Budynek",
    "technika budowy",
    "Wyposazenie",
    "Kuchnia",
    "Lazienka", 
    "Media",
    "Bezpieczenstwo",
    "Zrodlo",
    "Termin realizacji",
    "Status oferty",
    "Forma wlasnosci",
    "instalacje",
    "okna"
], inplace=True)

In [18]:
df["Dataset"] = "nieruchomosci"

In [19]:
df.columns

Index(['Miasto', 'URL', 'Cena', 'Powierzchnia', 'Pietro', 'Liczba pokoi',
       'Stan wykonczenia', 'Winda', 'Powierzchnia dodatkowa',
       'Miejsce parkingowe', 'Rynek', 'Rok budowy', 'Czynsz', 'latitude',
       'longitude', 'formatted_address', 'result_confidence', 'suburb',
       'building_category', 'result_type', 'pietro mieszkania',
       'maksymalne pietro', 'polozenie mieszkania', 'Dataset'],
      dtype='object')

In [20]:
df.shape

(27798, 24)

In [21]:
df.dtypes

Miasto                     object
URL                        object
Cena                      float64
Powierzchnia              float64
Pietro                     object
Liczba pokoi              float64
Stan wykonczenia           object
Winda                     float64
Powierzchnia dodatkowa     object
Miejsce parkingowe         object
Rynek                      object
Rok budowy                 object
Czynsz                    float64
latitude                  float64
longitude                 float64
formatted_address          object
result_confidence         float64
suburb                     object
building_category          object
result_type                object
pietro mieszkania         float64
maksymalne pietro         float64
polozenie mieszkania      float64
Dataset                    object
dtype: object

In [22]:
df.to_csv("nieruchomosci/nieruchomosci_online_before_merge.csv", index=False, sep="\t")